In [10]:
import pandas as pd
import os

# Rutas de los archivos de entrada
archivos = [
    "../data/Resultados_VA/cine_especifico/bogota_region/va_cine_especifico_2021_2022_nacionales_run18072025.xlsx",
    "../data/Resultados_VA/cine_especifico/bogota_region/va_cine_especifico_2022_2023_nacionales_run18072025.xlsx"
]

# Carpeta de salida y nombre del archivo final
carpeta_salida = "../data/Resultados_VA/cine_especifico/bogota_region/"
os.makedirs(carpeta_salida, exist_ok=True)
archivo_salida = os.path.join(carpeta_salida, "va_cine_especifico_x_programas_2021-2023_run18072025.xlsx")

# Leer y procesar ambos archivos
dfs = []
for archivo in archivos:
    df = pd.read_excel(archivo)
    df['snies_programas'] = df['snies_programas'].astype(str).str.replace(' ', '')
    df_exploded = df.assign(snies_programas=df['snies_programas'].str.split(';')).explode('snies_programas')
    df_exploded['snies_programas'] = df_exploded['snies_programas'].astype(int)
    dfs.append(df_exploded)

# Concatenar ambos DataFrames
df_concat = pd.concat(dfs, ignore_index=True)

# Crear tabla de presencia por periodo
presencia = df_concat.groupby(['snies_programas', 'periodo']).size().unstack(fill_value=0)

# Columna indicando si está en ambos periodos
presencia['en_ambos_periodos'] = (presencia > 0).sum(axis=1) == 2

# Unir esta información al DataFrame concatenado
df_final = df_concat.merge(
    presencia['en_ambos_periodos'].reset_index(),
    on='snies_programas',
    how='left'
)

# Guardar el resultado
df_final.to_excel(archivo_salida, index=False)
print(f"Archivo guardado en: {archivo_salida}")

Archivo guardado en: ../data/Resultados_VA/cine_especifico/bogota_region/va_cine_especifico_x_programas_2021-2023_run18072025.xlsx
